# simulation with ivs data

### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  

### Create graph

In [2]:
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph = dtv_backend.fis.load_fis_network(url)


### Make vessels and paths

In [3]:
data = gpd.read_file("data\ivs\ivs-2024-geocoded-sample.gpkg")
data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
data.dropna(subset = ['datetime', 'geometry'], inplace = True)
data = data.head()

D:\Users\ALPF\AppData\Local\Temp\ipykernel_26456\573042194.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



In [4]:
def closest_node_in_graph(graph, point):
    #get argument of closest node
    arg_min = np.argmin(shapely.distance(point, list(nx.get_node_attributes(graph, "geometry").values()) ))
    #get name of closest node
    node= list(graph.nodes)[arg_min]
    return node

In [14]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.Movable, 
                          core.HasResource, core.Routable,
                         core.ExtraMetadata), {})


vessels = []
for index, row in data.iterrows():
    #determine path
    point_1 = closest_node_in_graph(graph, Point(row.geometry.coords[0]) )
    point_2 = closest_node_in_graph(graph, Point(row.geometry.coords[-1]) )
    path = nx.dijkstra_path(graph, point_1, point_2)
    #determine capacity
    capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
    data_vessel = {"env": None,
               "name": row.name,
               "route": path,
               "geometry": Point(row.geometry.coords[0]),  # lon, lat
               "capacity": capacity,
               "v": 1
              }
    vessel = TransportResource(**data_vessel)
    vessel.name = "Vessel " + str(index + 1)
    vessels.append(vessel)

In [6]:
paths = []
for i in range(5):
    random_1 = random.choice(list(graph.nodes))
    random_2 = random.choice(list(graph.nodes))
    
    path = nx.dijkstra_path(graph, random_1, random_2)
    
    paths.append(path)

### Start simulation

In [7]:
def start(env, vessel):
    while True:
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [15]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())

# Add graph to environment
env.FG = graph

vessels = vessels[0:3]
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env
    vessel.geometry = nx.get_node_attributes(graph, "geometry")[paths[i][0]]

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))


env.epoch = datetime.datetime.now()

env.run()

### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [9]:
vessel_log = pd.DataFrame.from_dict(vessels[0].log)
vessel_log.head()

,Timestamp
0,2024-05-17 15:04:54.000000
1,2024-05-18 12:37:11.668044
2,2024-05-18 12:37:11.668044
3,2024-05-18 12:40:38.090775
4,2024-05-18 12:40:38.090775


### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [10]:
# visualise vessel movements based on the information included in the vessel.log
plot.vessel_kml(env, vessels, stepsize = 60)
plot.graph_kml(env)

KeyboardInterrupt: 